In [1]:
# install boto3 and fuzzysearch if not already installed
!pip install boto3
!pip install fuzzysearch

Processing /home/arjun/.cache/pip/wheels/fc/e6/eb/4c4b250a0d6562161dcb8667e2cb07a5e20b257fcb75e50b04/fuzzysearch-0.7.3-cp38-cp38-linux_x86_64.whl


'/home/arjun/Downloads'

In [1]:
# import packages
import boto3
import os
import spacy
import io
import cv2
import numpy as np
from PIL import Image as PImage
import re
from fuzzysearch import find_near_matches
import pandas as pd
import torch
import boto3
from IPython.display import Image, display
from PIL import Image as PImage, ImageDraw
import time
from __future__ import division
from __future__ import print_function
from scipy import sparse
import scipy.sparse as sp
import torch
import json
from scipy import sparse
import time
import argparse
import numpy as np
import cv2
from grapher import ObjectTree, Graph
import torch
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from utils import load_data, accuracy
from models import GCN
import os


import warnings
warnings.filterwarnings('ignore')



[nltk_data] Downloading package stopwords to /home/arjun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Curent AWS Region
mySession = boto3.session.Session(region_name='us-east-1')
awsRegion = mySession.region_name

In [3]:
# Amazon S3 client
s3 = boto3.client('s3')

# Amazon Textract client
textract = boto3.client('textract',region_name=awsRegion)

In [4]:
def add_values_in_dict(sample_dict, key, list_of_values):
    """Append multiple values to a key in the given dictionary"""
    if key not in sample_dict:
        sample_dict[key] = list()
    sample_dict[key].extend(list_of_values)
    return sample_dict

In [5]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [6]:
def spatial_test(df_,img_path): 
    import sys
    sys.argv=[''];
    del sys

    # Training settings
    parser = argparse.ArgumentParser()
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='Disables CUDA training.')
    parser.add_argument('--fastmode', action='store_true', default=True,
                        help='Validate during training pass.')
    parser.add_argument('--seed', type=int, default=69, help='Random seed.')
    parser.add_argument('--epochs', type=int, default=10000,
                        help='Number of epochs to train.')
    parser.add_argument('--lr', type=float, default=5e-3,
                        help='Initial learning rate.')
    parser.add_argument('--weight_decay', type=float, default=1e-4,
                        help='Weight decay (L2 loss on parameters).')
    parser.add_argument('--hidden', type=int, default=256,
                        help='Number of hidden units.')
    parser.add_argument('--dropout', type=float, default=0.5,
                        help='Dropout rate (1 - keep probability).')

    args = parser.parse_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)
    img = cv2.cvtColor(img_path, cv2.COLOR_BGR2GRAY)
    tree = ObjectTree()
    tree.read(df_, img)

    graph_dict, text_list = tree.connect(plot=True, export_df=True)

    graph = Graph(max_nodes=len(text_list))

    adj, features = graph.make_graph_data(graph_dict, text_list)
    new_feat=[]
    for rr in range(len(features)):
        new_feat.append(np.append(features[rr],(img.shape[0],img.shape[1])))
    features = np.array(new_feat)
    table_features = np.array(df_[['xmin','ymin','xmax','ymax','revised_distances_vert','revised_distances_hori']])
    features = np.concatenate((features,table_features),axis=1)
    ## adjacency matrix
    adj = sparse.csr_matrix(adj)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    adj = adj.todense()
    ## adjacency matrix to block diag
    from scipy.sparse import block_diag

    adj_array = []
    adj_array.append(adj)
    sparse_block_diag = block_diag(adj_array)


    values = sparse_block_diag.data
    indices = np.vstack((sparse_block_diag.row, sparse_block_diag.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = sparse_block_diag.shape

    sp_adj = torch.sparse.FloatTensor(i, v, torch.Size(shape)) # torch tensor adj matrix
    features = torch.FloatTensor(np.array(features))  # feature matrix for image
    return sp_adj,features,args

In [7]:
def test(df_,img_path):
    df_merged_labels = pd.read_csv('/home/arjun/same_set_df_merged_labels.csv')
    df_merged_labels.iloc[:, 13:] = df_merged_labels.iloc[:, 13:].fillna(0)
    labels = df_merged_labels.iloc[:, 13:]
    labels2 = torch.LongTensor(np.where(labels)[1])
    sp_adj,features,args = spatial_test(df_,img_path)
    model = GCN(nfeat=features.shape[1],
                nhid=args.hidden,
                nclass=labels2.max().item() + 1,
                dropout=args.dropout)
    model.load_state_dict(torch.load('/home/arjun/same_set_best_72_robert_40_text_int_22.pt'))
    #model.load_state_dict(torch.load('/media/arjun/Seagate Expansion Drive/gqp/features/'))

    model.eval()
    output = model(features, sp_adj)
    preds = output.max(1)[1].type_as(labels2)

    return preds,labels
    #loss_test = F.cross_entropy(output[idx_test], labels[idx_test])

    # print(output[idx_test],'output[idx_test]')
    # print(labels_df.columns)
    #print(labels[idx_test], 'labels[idx_test]')
    # writer.add_scalar('Loss/test', loss_test, epoch)

    #acc_test = accuracy(output[idx_test], labels[idx_test],flag=False,target_names= df_merged_labels.iloc[:, 13:].columns)
    #acc_train = accuracy(output[idx_train], labels[idx_train], flag=False,
                        #target_names=df_merged_labels.iloc[:, 13:].columns)


In [8]:
def test_spatial(df_,img_path='/home/arjun/PycharmProjects/pygcn/pygcn/test_files/train_img/final_train/receipt_00000.png'):
    preds,labels = test(df_,img_path)
    prediction = pd.DataFrame([np.array(preds.detach().cpu()),df_['Object']]).T
    prediction['col'] = prediction[0]
    for i in range(len(prediction)):
        prediction['col'][i] = labels.columns[prediction[0][i]]
    dict1 = {}
    for i in range(len(prediction)):
        add_values_in_dict(dict1,prediction['col'][i],[prediction[1][i]])
    
    return dict1

In [9]:
# function to call the spacy model and predict values and labels
def spacy_ner(text_in):
    # load model from google drive
    model_dir = '/home/arjun/Downloads/Spacy Model-CORD/Spacy Model/'
    
    nlp = spacy.load(model_dir)

    # generate predictions
    doc = nlp(text_in)

    # return predictions
    dicts = {ent.label_ : ent.text for ent in doc.ents}
    dict1={}
    for ent in doc.ents:
        add_values_in_dict(dict1,ent.label_,[ent.text])

    return dict1

In [10]:
 def call_textract(input):
    # reformat image into bytes
    try:
      imageBytes = bytearray(input.read())
    except:
      imageBytes = bytearray(input)

    # Call Amazon Textract
    decoded = cv2.imdecode(np.frombuffer(imageBytes, np.uint8), -1)
    #print(decoded.shape)
    response = textract.analyze_document(Document={'Bytes': imageBytes},FeatureTypes=['TABLES','FORMS'])
    imgWidth, imgHeight,channel = decoded.shape
    
    columns = ['xmin','ymin', 'xmax','ymax','Object']

    df_ = pd.DataFrame( columns=columns)
        
    # aggregate output into one document
    blocks=response['Blocks']
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block

    textfile = ''

    # Compile detected text
    for item in response["Blocks"]:
        if item["BlockType"] == 'WORD':
            textfile += item["Text"]+" "
            x_min = imgWidth*item['Geometry']['Polygon'][0]['X']
            x_max = imgWidth*item['Geometry']['Polygon'][1]['X']
            y_min = imgHeight*item['Geometry']['Polygon'][0]['Y']
            y_max = imgHeight*item['Geometry']['Polygon'][2]['Y']
            Object = item['Text']
            #print(x_min,'	',y_min,'	',x_max,'	',y_max,'	',item['Text'])

            df_ = df_.append({'xmin': x_min,'ymin':y_min,'xmax':x_max,'ymax':y_max,'Object':Object},ignore_index = True)

    return textfile,df_,decoded

In [11]:
# sharpen subprocess in preprocessing steps
def sharpen(image):
    kernel = np.array([[-1,-1,-1], 
                       [-1, 9,-1],
                       [-1,-1,-1]])
    sharpened = cv2.filter2D(image, -1, kernel) # applying the sharpening kernel to the input image & displaying it.
    return sharpened

# clahe subprocess in preprocessing steps
def clahe(img):
    #-----Converting image to LAB Color model----------------------------------- 
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)

    #-----Applying CLAHE to L-channel-----/--------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))

    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    return final

# preprocessing method 1
def cv_image_07(fullPath):
    image = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED);
    image = sharpen(image)
    image = cv2.GaussianBlur(image,(3,3),cv2.BORDER_DEFAULT)
    image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)  
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=400, sigma_r=0.2)
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=50, sigma_r=0.002)
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)
    gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    return gray

# preprocessing method 2
def cv_image_05(fullPath):
    image = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED);
    image = clahe(image)
    image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
    image = sharpen(image)
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=400, sigma_r=0.2)
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)
    gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    return gray

# preprocessing method 3
def cv_image_74_82(fullPath):
    image = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED);
    image = clahe(image)
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=400, sigma_r=0.2)
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0# Brightness control (0-100)
    gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    return gray

# preprocessing method 4
def cv_image_27(fullPath):
    image = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED);
    image = clahe(image)
    image = cv2.detailEnhance(image, sigma_s=150, sigma_r=0.15)
    image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=400, sigma_r=0.2)
    gray = cv2.fastNlMeansDenoisingColored(gray,None,10,10,7,21)
    return gray
    
# preprocessing method 5
def cv_image_11(fullPath):
    image = cv2.imread(fullPath, cv2.IMREAD_UNCHANGED);
    image = cv2.detailEnhance(image, sigma_s=200, sigma_r=0.05)
    gray = cv2.edgePreservingFilter(image, flags=1, sigma_s=400, sigma_r=0.2)
    gray = cv2.bitwise_not(gray)
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)
    gray = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    return gray

In [12]:
# reformats the image from the cv2 data type into a temp file bytes object that
# can be read by textract
def reformat(cv_img):
    # convert image to PIL format
    pil_im = PImage.fromarray(cv_img)
    # create temp file object
    img_byte_arr = io.BytesIO()
    # save image to temp file object as .png
    pil_im.save(img_byte_arr, format ='PNG')
    # retrieve values from temp image file
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

In [13]:
# master function that gradio calls
def textout(input,ppyn,method_option):   
    # if user selects no preprocessing
    if ppyn == "No":
        # call textract to return text
        nopp, df_,decoded = call_textract(input)
        # create label predictions from that text
        if method_option == 'Spacy':
            nopppred = spacy_ner(nopp)
        if method_option == 'Spatial':
            nopppred = test_spatial(df_,decoded)
        # convert image for display back to user
        pil_im = PImage.open(input.name)
        # return brand, label predictions and image back to user
        return nopppred, pil_im

    # if using preprocessing
    else:  
        # list of preprocessing functions
        funcs = [cv_image_07, cv_image_05, cv_image_74_82, cv_image_27, cv_image_11]
        # list of outputs from each preprocessing function
        lens = [] # length of label predictions
        preds = [] # label predictions
        imgs = [] # image used

        # first run with no preprocessing to get brand and predictions
        nopp,df_,decoded = call_textract(input)
        if method_option == 'Spatial':
            nopppred = test_spatial(df_,decoded)
        if method_option == 'Spacy':
            nopppred = spacy_ner(nopp)

        # append prediction, length of prediction, brand and image to lists
        preds.append(nopppred)
        lens.append(sum(len(x) for x in nopppred.values()))
        imgs.append(PImage.open(input.name))

        # iterate through preprocessing functions
        for f in funcs:
            # apply preprocessing
            pp = f(input.name)

            # run with preprocessed image to get brand and predictions
            pptext,df_,decoded = call_textract(reformat(pp))
            if method_option =='Spacy':
                pppred = spacy_ner(pptext)
            if method_option == 'Spatial':
                pppred = test_spatial(df_,decoded)


            # append prediction, length of prediction, brand and image to lists
            preds.append(pppred)
            lens.append(sum(len(x) for x in pppred.values()))
            imgs.append(pp)

        # calculate which method yields the longest predicted label
        biggest_output = lens.index(max(lens))

        # return to user the brand, label prediction and image used that yielded 
        # the longest predicted label
        final_pred = preds[biggest_output]
        final_img = PImage.fromarray(np.asarray(imgs[biggest_output]))

        # return brand, label predictions and image back to user
        return final_pred, final_img


Now we'll wrap this function with a Gradio interface.

In [14]:
import gradio

# image input function
image = gradio.inputs.Image(shape=None, source="upload", tool="select", type="file",label = "Nameplate Image")
# preprocessing yes/no radio button
pp_yn = gradio.inputs.Radio(choices=["Yes","No"],label = "Use Preprocessing on Image?")
method_option = gradio.inputs.Radio(choices=["Spatial","Spacy"],label = "Which method?")
# create interface that calls the textout function
gradio.Interface(textout, # function
    [image,pp_yn,method_option], # inputs
    [
      # outputs
      gradio.outputs.KeyValues(label="Predicted Values"),
      gradio.outputs.Image(type='pil',label="Image used for Predictions")    
    ]).launch(debug=False)

Running locally at: http://127.0.0.1:7865/
To get a public link for a hosted model, set Share=True
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7865/', None)